<a href="https://colab.research.google.com/github/misouza6828/ALURA-/blob/main/Transforma%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# === TRANSFORMAÇÃO — À PROVA DE FALHAS ===
import pandas as pd, numpy as np, json, requests, re
from pathlib import Path

# 1) helpers -------------------------------------------------------------
def to_dataframe(obj):
    if isinstance(obj, list):
        return pd.DataFrame(obj)
    if isinstance(obj, dict):
        for v in obj.values():
            if isinstance(v, list):
                return pd.DataFrame(v)
        return pd.json_normalize(obj)
    raise ValueError("formato inesperado")

def expand_dict_columns(df, sep="_"):
    out = df.copy()
    dict_cols = [c for c in out.columns if out[c].apply(lambda x: isinstance(x, dict)).any()]
    for col in dict_cols:
        exp = pd.json_normalize(out[col]).add_prefix(f"{col}{sep}")
        out = pd.concat([out.drop(columns=[col]), exp], axis=1)
    return out

def load_raw():
    # usa df_raw se já existir
    if 'df_raw' in globals():
        return df_raw
    # tenta JSON local
    if Path('TelecomX_Data.json').exists():
        with open('TelecomX_Data.json', 'r', encoding='utf-8') as f:
            raw = json.load(f)
        return to_dataframe(raw)
    # baixa da web (RAW da Alura)
    API_URL = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/main/TelecomX_Data.json"
    r = requests.get(API_URL, timeout=60); r.raise_for_status()
    return to_dataframe(r.json())

# 2) carrega o bruto (de onde der) --------------------------------------
df_raw = load_raw()

# 3) normaliza/expande dicionários --------------------------------------
df = expand_dict_columns(df_raw)

# 4) padroniza nomes
df.columns = (df.columns
              .str.strip()
              .str.replace(r"\s+","_", regex=True)
              .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
              .str.lower())

# 5) remove duplicados
df = df.drop_duplicates()

# 6) strip nas strings
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip()

# 7) converte numéricos principais (se existirem)
for col in ["account_chargesmonthly","account_chargestotal","customer_tenure","monthlycharges","totalcharges","tenure"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 8) trata nulos (numérico = mediana; texto = 'desconhecido')
for c in df.columns:
    if df[c].dtype.kind in "biufc":
        df[c] = df[c].fillna(df[c].median())
    else:
        df[c] = df[c].fillna("desconhecido")

# 9) mapeia churn para 0/1 (se existir)
if "churn" in df.columns:
    yes = {"yes","y","sim","true","1","churn","cancelou","evadiu"}
    no  = {"no","n","nao","não","false","0","ficou","ativo"}
    def _to01(v):
        s = str(v).strip().lower()
        if s in yes: return 1
        if s in no:  return 0
        try: return int(float(s))
        except: return np.nan
    df["churn"] = df["churn"].apply(_to01).fillna(0).astype(int)
else:
    print("⚠️ Coluna 'churn' não encontrada.")

# 10) features úteis
if "customer_tenure" in df.columns:
    df["tenure_years"] = (df["customer_tenure"]/12).round(2)
if "account_contract" in df.columns:
    df["contrato_mensal"] = df["account_contract"].astype(str).str.lower().str.contains("month").astype(int)
if "internet_internetservice" in df.columns:
    df["tem_fibra"] = df["internet_internetservice"].astype(str).str.lower().str.contains("fiber").astype(int)

print("Shape df_clean:", df.shape)
df.head()


Shape df_clean: (7267, 24)


,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_chargesmonthly,account_chargestotal,tenure_years,contrato_mensal,tem_fibra
0,0002-ORFBO,0,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,One year,Yes,Mailed check,65.6,593.30,0.75,0,0
1,0003-MKNFE,0,Male,0,No,No,9,Yes,Yes,DSL,...,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,0.75,1,0
2,0004-TLHLJ,1,Male,0,No,No,4,Yes,No,Fiber optic,...,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,0.33,1,1
3,0011-IGKFF,1,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,1.08,1,1
4,0013-EXCHZ,1,Female,1,Yes,No,3,Yes,No,Fiber optic,...,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,0.25,1,1
